In [2]:
import yfinance as yf
import datetime as dt
import pandas as pd
import numpy as np
from numpy import arange
import matplotlib.pyplot as plt
from pandas import read_csv

import ta as ta
from ta import add_all_ta_features
from ta.utils import dropna
from datetime import datetime

from tsmoothie.utils_func import sim_randomwalk
from tsmoothie.smoother import *

from sklearn.preprocessing import MinMaxScaler

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

from sklearn.metrics import mean_absolute_error

import joblib

import time

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

def preparar_dados_para_treinamento(anteriores,base_treinamento_normalizada):

    previsores = []
    preco_real = []

    for i in range(anteriores,len(base_treinamento_normalizada)):

        previsores.append(base_treinamento_normalizada[i-anteriores:i,0])
        preco_real.append(base_treinamento_normalizada[i,0])

    previsores,preco_real = np.array(previsores),np.array(preco_real)
    previsores = previsores
    
    return previsores,preco_real

def GetTsi(base,gaussian_knots,gaussian_sigma,ewm_span=20):
    
    tsi_config=[25,13]

    resultados_tsi = ta.momentum.TSIIndicator(base["Close"],tsi_config[0],tsi_config[1],False)

    tsi_df = pd.DataFrame(resultados_tsi.tsi())
    
    tsi_df.dropna(inplace=True)
    
    #Suavizando TSI com médias móveis exponenciais
    tsi_df["ewm"] = tsi_df['tsi'].ewm(span = ewm_span).mean()*1.2
    #------------------------------------------
    
    #Suavizanto TSI com gaussian smoother
    tsi_np = tsi_df["tsi"].to_numpy()
    tsi_np.reshape(1,len(tsi_np))

    smoother = GaussianSmoother(n_knots=gaussian_knots, sigma=gaussian_sigma)
    smoother.smooth(tsi_np)

    tsi_df["gaussian"] = smoother.smooth_data[0]
    #------------------------------------------
    
    return tsi_df

def GetRoc(base,gaussian_knots,gaussian_sigma,ewm_span=20):
    
    roc_config=[12]

    resultados_roc = ta.momentum.ROCIndicator(base["Close"],roc_config[0],False)
    
    roc_df = pd.DataFrame(resultados_roc.roc(),columns=["roc"])
    
    roc_df.dropna(inplace=True)
    
    #Suavizando ROC com médias móveis exponenciais
    roc_df["ewm"] = roc_df['roc'].ewm(span = ewm_span).mean()*1.2
    
    #Suavizanto ROC com gaussian smoother
    roc_np = roc_df["roc"].to_numpy()
    roc_np.reshape(1,len(roc_np))

    smoother = GaussianSmoother(n_knots=gaussian_knots, sigma=gaussian_sigma)
    smoother.smooth(roc_np)

    roc_df["gaussian"] = smoother.smooth_data[0]
    #------------------------------------------
    
    return roc_df

def Normalizar(Oscilador,coluna):
    
    normalizador = MinMaxScaler(feature_range=(0,1))
    
    if coluna == "tsi":
        Oscilador_treinamento = Oscilador.iloc[:,0:1].values
        
    if coluna == "ewm":
        Oscilador_treinamento = Oscilador.iloc[:,1:2].values
        
    if coluna == "gaussian":
        Oscilador_treinamento = Oscilador.iloc[:,2:3].values
        
    Oscilador_normalizado = normalizador.fit_transform(Oscilador_treinamento)
    
    return Oscilador_normalizado



In [ ]:
ticker ="PETR3.SA"

df = yf.download(ticker)

base = df["2018":]


fig = go.Figure()

fig.add_trace(
    go.Scatter(
        name="Ação",
        x=base.index,
        y=base["Close"],
        line=dict(color='blue', width=1))
)

In [ ]:
#TSI

tsi_config=[25,13]
resultados_tsi = ta.momentum.TSIIndicator(base["Close"],tsi_config[0],tsi_config[1],False)

resultados_tsi.tsi().dropna(inplace=True)


#Gaussian TSI
resultados_tsi_sem_null = resultados_tsi.tsi().dropna()

tsi_np = resultados_tsi_sem_null.to_numpy()

tsi_np = tsi_np.reshape(1,len(tsi_np))


smoother_tsi = GaussianSmoother(n_knots=90, sigma=0.0003)
smoother_tsi.smooth(tsi_np)


fig = go.Figure()

fig.add_trace(
    go.Scatter(
        name="TSI",
        x=base.tail(len(resultados_tsi_sem_null)).index,
        y=resultados_tsi_sem_null,
        line=dict(color='blue', width=1))
)

fig.add_trace(
    go.Scatter(
        x=base.tail(len(resultados_tsi_sem_null)).index,
        y=smoother_tsi.smooth_data[0],
        mode='lines',
        name='Gaussian',
        line=dict(color="orange"))
)


fig.add_hline(y=0,line_color="blue",line_width=1, line_dash="dash")
fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')

In [31]:
ticker ="PETR3.SA"

df = yf.download(ticker)

base = df["2018":]


knots = 60
n_sigma = 0.0003

anteriores_ = 90


tsi = GetTsi(base,knots,n_sigma)
#--------------------------------


#Faz a normalização do gaussian do tsi
normalizado = Normalizar(tsi,"tsi")
#--------------------------------

#Oscilador_treinamento = tsi.iloc[:,0:1].values


X_train, y_train = preparar_dados_para_treinamento(anteriores_,normalizado)


[*********************100%***********************]  1 of 1 completed


In [30]:
model = GaussianNB()

model.fit(X_train,y_train)

ValueError: Unknown label type: (array([-51.55802628, -51.48633914, -50.51826063, ...,  50.17783653,
        52.42723708,  52.99686056]),)